In [ ]:
from google.colab import drive
drive.mount('/content/drive')

json >> yolo형식에 맞는 txt형식으로 변환

class id x y width height

In [ ]:
import json
import os
import glob

# JSON 파일들이 있는 디렉토리
json_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/val/json'  # JSON 파일들이 있는 디렉토리 경로
# YOLO 형식 라벨을 저장할 디렉토리
labels_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/val/labels'  # YOLO 라벨을 저장할 디렉토리 경로

# 클래스 이름을 클래스 ID로 매핑
class_name_to_id = {
    0: 0,  # 얼굴 전체
    1: 1,  # 이마
    2: 2,  # 미간
    3: 3,  # 왼쪽 눈가
    4: 4,  # 오른쪽 눈가
    5: 5,  # 왼쪽 볼
    6: 6,  # 오른쪽 볼
    7: 7,  # 입술
    8: 8,  # 턱
}

# YOLO 형식으로 변환 함수
def convert_to_yolo_format(image_width, image_height, bbox):
    if bbox is None or len(bbox) != 4:
        raise ValueError("Invalid bounding box format. Expected [x_min, y_min, width, height].")
    x_min, y_min, x_max, y_max = bbox

    # x_min = max(0, min(x_min, image_width))
    # y_min = max(0, min(y_min, image_height))
    # x_max = max(0, min(x_min + width, image_width))
    # y_max = max(0, min(y_min + height, image_height))

    width = (x_max - x_min)
    height = (y_max - y_min)

    x_center = ((x_min + width / 2)) / image_width
    y_center = ((y_min + height / 2)) / image_height
    width /= image_width
    height /= image_height
    return x_center, y_center, width, height

# YOLO 라벨 저장 디렉토리가 없으면 생성
os.makedirs(labels_dir, exist_ok=True)

# 각 각도에 대한 통합된 YOLO 파일 생성
angle_to_image_labels = {0: {}, 7: {}, 8: {}}

# JSON 파일들을 순회
for json_file_path in glob.glob(os.path.join(json_dir, '*.json')):
    # JSON 파일 읽기
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # 이미지 메타데이터 추출
    image_info = data['info']
    image_filename = image_info['filename']
    image_width = data['images']['width']
    image_height = data['images']['height']
    bbox = data['images'].get('bbox', None)  # bbox가 None일 수 있으므로 예외 처리
    angle = data['images']['angle']
    facepart = data['images']['facepart']

    # 클래스 ID 추출
    class_id = class_name_to_id.get(facepart, None)

    # 바운딩 박스를 YOLO 형식으로 변환
    if class_id is not None and bbox is not None and angle in angle_to_image_labels:
        try:
            x_center, y_center, width, height = convert_to_yolo_format(image_width, image_height, bbox)
            yolo_label = f"{class_id} {x_center} {y_center} {width} {height}\n"

            # 각 각도에 대해 이미지 라벨 통합
            if image_filename not in angle_to_image_labels[angle]:
                angle_to_image_labels[angle][image_filename] = []
            angle_to_image_labels[angle][image_filename].append(yolo_label)
        except ValueError as e:
            print(f"Error processing file {json_file_path}: {e}")

# 각 각도에 대한 YOLO 라벨 파일 작성
for angle, images in angle_to_image_labels.items():
    for image_filename, labels in images.items():
        yolo_label_path = os.path.join(labels_dir, f"{os.path.splitext(image_filename)[0]}.txt")
        with open(yolo_label_path, 'w') as label_file:
            label_file.writelines(labels)

        print(f"Saved YOLO label to {yolo_label_path}")


In [ ]:
# %cd /content/drive/MyDrive/최종프로젝트/이미지 처리
# !git clone https://github.com/ultralytics/yolov5

In [ ]:
# %cd /content/drive/MyDrive/최종프로젝트/이미지 처리/yolov5
# !pip install -r requirements.txt

In [ ]:
%cat /content/drive/MyDrive/Final_project/image_detect/dataset/data.yaml

모델 학습

https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

In [ ]:
%cd /content/drive/MyDrive/Final_project/image_detect/yolov5
!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/Final_project/image_detect/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name team2_yolov5s_results

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Final_project/image_detect/yolov5/runs/

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt

# YOLOv5 모델 로드 (학습된 가중치 사용)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Final_project/image_detect/yolov5/runs/train/team2_yolov5s_results/weights/best.pt')

# 이미지 경로 설정
image_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/myface.jpg'  # 테스트할 이미지 파일 경로

# 이미지 로드 (OpenCV 사용)
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image not found or unable to open: {image_path}")

# 이미지 크기 조정
new_width, new_height = 480, 600
resized_image = cv2.resize(image, (new_width, new_height))

# BGR 이미지를 RGB로 변환
image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

# 이미지 탐지
results = model(image_rgb)

# 결과 시각화
results.print()  # 탐지 결과 출력
results.show()   # 탐지된 이미지 시각화

# 결과 이미지 저장
output_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/save_image/label_image.jpg'
cv2.imwrite(output_path, image_rgb)

# 이미지 출력 (matplotlib 사용)
plt.imshow(image_rgb)
plt.axis('off')
plt.show()



In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np

# YOLOv5 모델 로드 (학습된 가중치 사용)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Final_project/image_detect/yolov5/runs/train/team2_yolov5s_results/weights/best.pt')

# 이미지 경로 설정
image_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/myface.jpg'  # 테스트할 이미지 파일 경로

# 이미지 로드 (OpenCV 사용)
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image not found or unable to open: {image_path}")

# 이미지 크기 조정
new_width, new_height = 480, 600
resized_image = cv2.resize(image, (new_width, new_height))

# BGR 이미지를 RGB로 변환
image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

# 이미지 탐지
results = model(image_rgb)

# 결과 시각화
results.print()  # 탐지 결과 출력
results.show()   # 탐지된 이미지 시각화

# 바운딩 박스와 클래스 ID 추출 및 저장
detection_data = results.xyxy[0].cpu().numpy()  # xyxy 포맷의 결과를 numpy 배열로 변환
output_txt_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/save_bbox/label_image.txt'

with open(output_txt_path, 'w') as f:
    for *box, conf, cls in detection_data:
        x1, y1, x2, y2 = map(int, box)  # 바운딩 박스 좌표를 정수로 변환
        class_id = int(cls)
        f.write(f"{class_id} {x1} {y1} {x2} {y2}\n")

# 결과 이미지 저장
output_image_path = '/content/drive/MyDrive/Final_project/image_detect/dataset/save_image/label_image.jpg'
cv2.imwrite(output_image_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))

# 이미지 출력 (matplotlib 사용)
plt.imshow(image_rgb)
plt.axis('off')
plt.show()


In [ ]:
import os

label_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/val/labels'

# 레이블 파일들 확인
for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), 'r') as file:
        for line in file:
            values = line.strip().split()
            if len(values) != 5:
                print(f"Error in file {label_file}: {line}")

1이 넘어가는 값이 있는지 확인

In [ ]:
import os

label_dir = '/content/drive/MyDrive/Final_project/image_detect/dataset/val/labels'

for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), 'r') as file:
        for line in file:
            values = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, values)
            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and
                    0 <= width <= 1 and 0 <= height <= 1):
                print(f"Out of bounds in file {label_file}: {line}")
